In [220]:
grid = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]

In [222]:
from IPython.display import HTML, display

def display_colored_grid(grid):
    html = '<table style="border-collapse: collapse;">'
    
    for row in grid:
        html += '<tr>'
        for cell in row:
            html += f'''
            <td style="
                width: 30px; 
                height: 30px;
                background-color: white;
                border: 1px solid gray;
                text-align: center;
                vertical-align: middle;
                font-family: Arial;
                font-size: 12px;
            ">{cell}</td>
            '''
        html += '</tr>'
    html += '</table>'
    
    display(HTML(html))

In [224]:
display_colored_grid(grid)

In [226]:
def validate_cell(cell):
    x, y = cell
    return 0 <= x <= 2 and 0 <= y <= 2

In [228]:
zero = (0, 0)

In [230]:
import random

def randomize_grid(grid, zero, moves=5):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    
    for _ in range(moves):
        random.shuffle(directions)
        for direction in directions:
            new_cell = (zero[0] + direction[0], zero[1] + direction[1])
            if validate_cell(new_cell):
                x, y = zero
                new_x, new_y = new_cell
                grid[x][y], grid[new_x][new_y] = grid[new_x][new_y], grid[x][y]
                zero = new_cell
                break
    
    return zero


In [232]:
zero = randomize_grid(grid, zero)

In [234]:
display_colored_grid(grid)

In [236]:
import heapq
from IPython.display import HTML, display

# Define goal state
GOAL_STATE = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]

# Possible moves for blank space (zero)
MOVES = {
    "UP": (-1, 0),
    "DOWN": (1, 0),
    "LEFT": (0, -1),
    "RIGHT": (0, 1)
}

def display_colored_grid(grid):
    html = '<table style="border-collapse: collapse;">'
    
    for row in grid:
        html += '<tr>'
        for cell in row:
            html += f'''
            <td style="
                width: 30px; 
                height: 30px;
                background-color: white;
                border: 1px solid gray;
                text-align: center;
                vertical-align: middle;
                font-family: Arial;
                font-size: 12px;
            ">{cell}</td>
            '''
        html += '</tr>'
    html += '</table>'
    
    display(HTML(html))

class PuzzleState:
    def __init__(self, grid, zero_pos, g, parent=None):
        self.grid = grid  
        self.zero_pos = zero_pos 
        self.g = g  # Cost from start state
        self.parent = parent  # Parent state for backtracking path

    def __lt__(self, other):
        """ Needed for priority queue sorting based on f(n) """
        return (self.g + self.heuristic()) < (other.g + other.heuristic())

    def heuristic(self):
        """ Manhattan distance heuristic function """
        distance = 0
        for r in range(3):
            for c in range(3):
                value = self.grid[r][c]
                if value != 0:  # Skip the blank tile
                    goal_r, goal_c = divmod(value - 1, 3)  # Correct position
                    distance += abs(goal_r - r) + abs(goal_c - c)
        return distance

    def get_next_states(self):
        """ Generate possible next states """
        next_states = []
        x, y = self.zero_pos

        for move, (dx, dy) in MOVES.items():
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                # Swap tiles
                new_grid = [row[:] for row in self.grid]
                new_grid[x][y], new_grid[new_x][new_y] = new_grid[new_x][new_y], new_grid[x][y]
                next_states.append(PuzzleState(new_grid, (new_x, new_y), self.g + 1, self))
        
        return next_states

    def is_goal(self):
        """ Check if current state is the goal state """
        return self.grid == GOAL_STATE

def solve_puzzle(initial_grid):
    """ Solves the 8-puzzle using A* """
    zero_pos = next((r, c) for r in range(3) for c in range(3) if initial_grid[r][c] == 0)
    start_state = PuzzleState(initial_grid, zero_pos, 0)
    
    # Priority queue for A* search
    open_set = []
    heapq.heappush(open_set, start_state)
    
    visited = set()
    visited.add(tuple(map(tuple, start_state.grid)))

    while open_set:
        current = heapq.heappop(open_set)

        if current.is_goal():
            path = []
            while current:
                path.append(current.grid)
                current = current.parent
            return path[::-1]  # Return path from start to goal

        for next_state in current.get_next_states():
            grid_tuple = tuple(map(tuple, next_state.grid))
            if grid_tuple not in visited:
                heapq.heappush(open_set, next_state)
                visited.add(grid_tuple)
    
    return None  # No solution found (shouldn't happen for a solvable puzzle)

# Example: Initial state
initial_state = grid

# Solve the puzzle
solution_path = solve_puzzle(initial_state)

# Print the solution steps
if solution_path:
    for step, state in enumerate(solution_path):
        print(f"Step {step}:")
        display_colored_grid(state)
        print("\n")
else:
    print("No solution found!")

Step 0:




Step 1:




Step 2:




Step 3:
